# Plan: SETI Breakthrough Listen (MLE-Benchmark) — Medal-Oriented Workflow

Objectives:
- Establish a fast, correct baseline and robust CV that mirrors test.
- Leverage GPU for modeling (likely CNN/1D-CNN on waterfall/spectrogram .npy, and/or fast GBMs on engineered features).
- Iterate with OOF-driven improvements and simple ensembling.

Data understanding (expected):
- train/: signal files (likely .npy per id).
- test/: same format without labels.
- train_labels.csv: id,label mapping.
- sample_submission.csv: required submission format.
- old_leaky_data/: ignore for leakage safety unless validated by experts.

Validation strategy:
- Stratified KFold by label (5 folds) with deterministic seed.
- No leakage from file ordering/neighboring samples. Fit transforms within fold only.

Baseline roadmap:
1) Environment & GPU check; torch install (CUDA 12.1 wheels), or use XGBoost/CatBoost GPU.
2) EDA: verify counts, file presence per id, shapes/dtypes; quick visualization of a few signals.
3) Fast baseline A: Simple feature extraction from arrays (stats across axes, spectrum bands) → XGBoost GPU. Target: achieve >0.74 AUC quickly.
4) Baseline B: Lightweight CNN on downsampled spectrograms (PyTorch, mixed precision). Target: >0.76 AUC.
5) Improve features: domain-inspired features (band energies, slopes, peaks, kurtosis, Hough-like line strength proxies).
6) Tune and ensemble: blend GBM + CNN logits; calibrate if needed. Target: ≥0.775 (bronze) then push towards ≥0.781 (silver).

Operational rules:
- Subsample first (e.g., 10k rows, 2 folds) for smoke tests. Log elapsed times.
- Cache features and OOF/test logits in .npy. Reuse for blends.
- One change per run; track deltas.
- Avoid old_leaky_data unless experts confirm a safe use.

Next steps this notebook:
A) Run GPU and repo sanity checks; list files.
B) Load labels, verify id↔file mapping.
C) Implement minimal feature extractor v0 and XGBoost GPU baseline with 5-fold CV.
D) Save OOF/test preds; create submission.csv and verify format.

Checkpoints for expert review:
- After this plan (now).
- After data loading/EDA and CV setup.
- After baseline GBM OOF results.
- After CNN baseline.
- Before final blend and submission.

In [1]:
# Environment and data sanity checks
import os, sys, time, json, glob, random, math, shutil, subprocess, textwrap, gc
from pathlib import Path
import numpy as np
import pandas as pd

start_time = time.time()
def log(msg):
    print(f"[{time.strftime('%H:%M:%S')}] {msg}", flush=True)

# 1) GPU check
log('Checking GPU with nvidia-smi...')
try:
    out = subprocess.run(['bash','-lc','nvidia-smi || true'], capture_output=True, text=True, check=False)
    print(out.stdout)
except Exception as e:
    log(f'GPU check failed: {e}')

# 2) List files
base = Path('.')
log('Listing key files...')
print('CWD:', base.resolve())
for p in ['train_labels.csv','sample_submission.csv','train','test','old_leaky_data']:
    pp = base / p
    print(f"- {p} exists? {pp.exists()} type: {'dir' if pp.is_dir() else 'file' if pp.exists() else 'NA'}")

# 3) Load labels and sample submission
lbl_path = base/'train_labels.csv'
sub_path = base/'sample_submission.csv'
assert lbl_path.exists(), 'train_labels.csv missing'
assert sub_path.exists(), 'sample_submission.csv missing'

train_labels = pd.read_csv(lbl_path)
sample_sub = pd.read_csv(sub_path)
log('train_labels head:')
print(train_labels.head())
log('sample_submission head:')
print(sample_sub.head())

# Basic label stats
log('Label distribution:')
print(train_labels['target'].value_counts(normalize=False).to_string())
print(train_labels['target'].value_counts(normalize=True).round(4).to_string())

# 4) Discover npy files
train_dir = base/'train'
test_dir = base/'test'
train_files = sorted(glob.glob(str(train_dir/'**'/'*.npy'), recursive=True))
test_files = sorted(glob.glob(str(test_dir/'**'/'*.npy'), recursive=True))
log(f'Found {len(train_files)} train npy files and {len(test_files)} test npy files')
print('First 5 train files:', train_files[:5])
print('First 5 test files:', test_files[:5])

# 5) Verify id<->file mapping (assumes filename stem is id)
def stem(p):
    return Path(p).stem
train_stems = set(map(stem, train_files))
test_stems = set(map(stem, test_files))
ids_in_labels = set(train_labels['id'].astype(str).values)
missing_files = ids_in_labels - train_stems
extra_files = train_stems - ids_in_labels
log(f'IDs in labels: {len(ids_in_labels)} | unique train file stems: {len(train_stems)}')
log(f'Missing file for {len(missing_files)} ids (show up to 5): {list(sorted(missing_files))[:5]}')
log(f'Extra train files without labels: {len(extra_files)} (show up to 5): {list(sorted(extra_files))[:5]}')

# 6) Inspect shape/dtype of a few files
def inspect_file(fp):
    arr = np.load(fp, allow_pickle=False, mmap_mode=None)
    return arr.shape, arr.dtype, np.nanmin(arr), np.nanmax(arr), np.nanmean(arr)

for fp in random.sample(train_files, k=min(3, len(train_files))):
    try:
        shape, dtype, vmin, vmax, vmean = inspect_file(fp)
        log(f'Inspect {Path(fp).name}: shape={shape}, dtype={dtype}, min={vmin:.4f}, max={vmax:.4f}, mean={vmean:.4f}')
    except Exception as e:
        log(f'Failed to load {fp}: {e}')

# 7) Quick check: ensure sample_submission ids match test stems
sub_ids = set(sample_sub['id'].astype(str).values)
missing_test_for_sub = sub_ids - test_stems
log(f'sample_submission ids: {len(sub_ids)}; test stems: {len(test_stems)}; missing in test for sub: {len(missing_test_for_sub)}')
if missing_test_for_sub:
    print('Example missing ids:', list(sorted(missing_test_for_sub))[:5])

log(f'Done in {time.time()-start_time:.2f}s')

# Notes for next steps:
# - If shapes are (6,H,W), proceed with 6-ch pipeline (diffs + off channels).
# - Save discovered mapping and stats for reuse.

with open('data_discovery.json','w') as f:
    json.dump({
        'n_train_files': len(train_files),
        'n_test_files': len(test_files),
        'n_label_ids': len(ids_in_labels),
        'missing_files_count': len(missing_files),
        'extra_files_count': len(extra_files)
    }, f, indent=2)

[23:32:35] Checking GPU with nvidia-smi...


Wed Sep 24 23:32:35 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.144.06             Driver Version: 550.144.06     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A10-24Q                 On  |   00000002:00:00.0 Off |                    0 |
| N/A   N/A    P0             N/A /  N/A  |     182MiB /  24512MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

CWD: /var/lib/simon/agent_run_states/seti-breakthrough-listen-20250924-194856
- train_labels.csv exists? True type: file
- sample_submission.csv exists? True type: file
- train exists? True type: dir
- test exists? True type: dir
- old_leaky_data exists? True type: dir
[23:32:35] train_labels head:


                id  target
0  d5d85dafc41d5b3       0
1  6170c3d29bd5874       0
2  87989f418ca1301       0
3  3087c24fbcb2c3b       0
4  8b04fea0d8d49c8       0
[23:32:35] sample_submission head:


                id  target
0  0cee567456cd304     0.5
1  5451b45281c65a7     0.5
2  f8cc6cea820282d     0.5
3  25e21ba81a64742     0.5
4  aafa910406b1db2     0.5
[23:32:35] Label distribution:


...

[23:32:35] Inspect 2870a66a0120636.npy: shape=(6, 273, 256), dtype=float16, min=-0.6470, max=15.8984, mean=-0.0000


[23:32:35] Inspect 540cb12920f8d8b.npy: shape=(6, 273, 256), dtype=float16, min=-0.7539, max=15.9219, mean=0.0000


[23:32:35] Inspect 13a464f77b72d52.npy: shape=(6, 273, 256), dtype=float16, min=-0.7764, max=16.0000, mean=-0.0000


[23:32:35] sample_submission ids: 6000; test stems: 6000; missing in test for sub: 0


[23:32:35] Done in 0.29s


In [2]:
# Install PyTorch cu121 stack and deps, then sanity-check GPU
import os, sys, subprocess, shutil, time
from pathlib import Path

start = time.time()
def pip(*args):
    print('> pip', *args, flush=True)
    subprocess.run([sys.executable, '-m', 'pip', *args], check=True)

# 0) Uninstall any pre-existing torch stacks to avoid conflicts
for pkg in ('torch','torchvision','torchaudio'):
    subprocess.run([sys.executable, '-m', 'pip', 'uninstall', '-y', pkg], check=False)

# Clean stray site dirs that can shadow correct wheels (idempotent)
for d in (
    '/app/.pip-target/torch',
    '/app/.pip-target/torch-2.8.0.dist-info',
    '/app/.pip-target/torch-2.4.1.dist-info',
    '/app/.pip-target/torchvision',
    '/app/.pip-target/torchvision-0.23.0.dist-info',
    '/app/.pip-target/torchvision-0.19.1.dist-info',
    '/app/.pip-target/torchaudio',
    '/app/.pip-target/torchaudio-2.8.0.dist-info',
    '/app/.pip-target/torchaudio-2.4.1.dist-info',
    '/app/.pip-target/torchgen',
    '/app/.pip-target/functorch',
):
    if os.path.exists(d):
        print('Removing', d); shutil.rmtree(d, ignore_errors=True)

# 1) Install EXACT cu121 torch stack
pip('install',
    '--index-url', 'https://download.pytorch.org/whl/cu121',
    '--extra-index-url', 'https://pypi.org/simple',
    'torch==2.4.1', 'torchvision==0.19.1', 'torchaudio==2.4.1')

# 2) Freeze torch versions for later installs
Path('constraints.txt').write_text('torch==2.4.1\ntorchvision==0.19.1\ntorchaudio==2.4.1\n')

# 3) Install non-torch deps honoring constraints
pip('install', '-c', 'constraints.txt',
    'timm==1.0.9', 'scikit-learn', 'numpy', 'pandas',
    '--upgrade-strategy', 'only-if-needed')

# 4) Sanity gate
import torch
print('torch:', torch.__version__, 'built CUDA:', getattr(torch.version, 'cuda', None))
print('CUDA available:', torch.cuda.is_available())
assert str(getattr(torch.version,'cuda','')).startswith('12.1'), f'Wrong CUDA build: {torch.version.cuda}'
assert torch.cuda.is_available(), 'CUDA not available'
print('GPU:', torch.cuda.get_device_name(0))
torch.backends.cudnn.benchmark = True
print(f'Install+sanity completed in {time.time()-start:.1f}s')

> pip install --index-url https://download.pytorch.org/whl/cu121 --extra-index-url https://pypi.org/simple torch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1


Looking in indexes: https://download.pytorch.org/whl/cu121, https://pypi.org/simple


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 799.0/799.0 MB 282.2 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 391.2 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 535.8 MB/s eta 0:00:00


...

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.7/39.7 MB 468.8 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 226.1 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 KB 500.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.2/161.2 KB 462.8 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.8/129.8 KB 480.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 KB 451.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 KB 242.3 MB/s eta 0:00:00


torch: 2.4.1+cu121 built CUDA: 12.1
CUDA available: True
GPU: NVIDIA A10-24Q
Install+sanity completed in 132.5s


In [3]:
# Install xgboost GPU and run GBM feature baseline
import sys, subprocess, time, json
from pathlib import Path

def run(cmd):
    print('> ', ' '.join(cmd), flush=True)
    return subprocess.run(cmd, check=True)

t0 = time.time()
try:
    import xgboost as xgb
    print('xgboost already installed:', xgb.__version__)
except Exception:
    run([sys.executable, '-m', 'pip', 'install', 'xgboost==2.1.1'])
    import xgboost as xgb
    print('xgboost installed:', xgb.__version__)

# Launch XGB training (features + 5-fold CV). Logs progress every 1000 rows.
print('Starting XGBoost v0 features run...', flush=True)
ret = subprocess.run([sys.executable, 'train_xgb.py', '--out_dir', 'outputs_xgb', '--folds', '5', '--seed', '42'])
print('XGB run return code:', ret.returncode, 'elapsed:', round(time.time()-t0,1), 's', flush=True)

xgboost already installed: 2.1.4
Starting XGBoost v0 features run...


Found train files: 54000, test files: 6000


Featurizing train...
  featurizing 1000/54000 elapsed 48.8s


  featurizing 2000/54000 elapsed 98.0s


  featurizing 3000/54000 elapsed 147.0s


...

  featurizing 13000/54000 elapsed 639.2s


  featurizing 14000/54000 elapsed 688.0s


  featurizing 15000/54000 elapsed 736.9s


  featurizing 16000/54000 elapsed 786.1s


  featurizing 17000/54000 elapsed 835.3s


In [6]:
# Run CNN sanity check: overfit_small with forced pairing B
import sys, subprocess, time

cmd = [sys.executable, 'train_cnn.py',
       '--overfit_small',
       '--force_pairing', 'B',
       '--batch_size', '96',
       '--epochs', '8',
       '--image_size', '256',
       '--num_workers', '8',
       '--out_dir', 'outputs_cnn',
       '--seed', '42']
print('> ', ' '.join(cmd), flush=True)
t0 = time.time()
ret = subprocess.run(cmd)
print('Return code:', ret.returncode, 'elapsed:', round(time.time()-t0,1), 's', flush=True)

>  /usr/bin/python3.11 train_cnn.py --overfit_small --force_pairing B --batch_size 96 --epochs 8 --image_size 256 --num_workers 8 --out_dir outputs_cnn --seed 42


Device: cuda


Found train files: 54000, test files: 6000
Using force_pairing=B
--- Fold 1/1 ---


Fold 0 Epoch 1/8 lr=0.000200


Fold 0 Epoch 1: train_loss=0.6942 val_auc=0.53194
Fold 0 Epoch 2/8 lr=0.000951


...

Fold 0 Epoch 6: train_loss=0.4029 val_auc=0.99809
Fold 0 Epoch 7/8 lr=0.000059


Fold 0 Epoch 7: train_loss=0.3633 val_auc=0.99931
Fold 0 Epoch 8/8 lr=0.000010


Fold 0 Epoch 8: train_loss=0.3540 val_auc=0.99953
Fold 0 best AUC: 0.99953 saved to outputs_cnn/model_fold0.pt


Traceback (most recent call last):
  File "/var/lib/simon/agent_run_states/seti-breakthrough-listen-20250924-194856/train_cnn.py", line 442, in <module>
    main()
  File "/var/lib/simon/agent_run_states/seti-breakthrough-listen-20250924-194856/train_cnn.py", line 417, in main
    oof[val_idx] = val_preds.astype(np.float32)
    ~~~^^^^^^^^^
ValueError: shape mismatch: value array of shape (512,) could not be broadcast to indexing result of shape (54000,)


Return code: 1 elapsed: 102.1 s


In [7]:
# Run full 5-fold CNN CV with forced pairing B
import sys, subprocess, time

cmd = [sys.executable, 'train_cnn.py',
       '--batch_size', '96',
       '--epochs', '12',
       '--image_size', '256',
       '--num_workers', '8',
       '--out_dir', 'outputs_cnn_full',
       '--seed', '42',
       '--force_pairing', 'B']
print('> ', ' '.join(cmd), flush=True)
t0 = time.time()
ret = subprocess.run(cmd)
print('Return code:', ret.returncode, 'elapsed:', round(time.time()-t0,1), 's', flush=True)

>  /usr/bin/python3.11 train_cnn.py --batch_size 96 --epochs 12 --image_size 256 --num_workers 8 --out_dir outputs_cnn_full --seed 42 --force_pairing B


Device: cuda


Found train files: 54000, test files: 6000
Using force_pairing=B
--- Fold 1/5 ---


Fold 0 Epoch 1/12 lr=0.000200


  iter 100/450 loss=0.5756 elapsed=13.2s


...

  iter 300/450 loss=0.5295 elapsed=7.6s


  iter 400/450 loss=0.5273 elapsed=7.5s


Fold 0 Epoch 4: train_loss=0.5254 val_auc=0.48535
Early stopping at epoch 4
Fold 0 best AUC: 0.50746 saved to outputs_cnn_full/model_fold0.pt


Traceback (most recent call last):
  File "/var/lib/simon/agent_run_states/seti-breakthrough-listen-20250924-194856/train_cnn.py", line 448, in <module>
    main()
  File "/var/lib/simon/agent_run_states/seti-breakthrough-listen-20250924-194856/train_cnn.py", line 418, in main
    _, val_preds, _ = valid_one_epoch(model, valid_loader, device)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/lib/simon/agent_run_states/seti-breakthrough-listen-20250924-194856/train_cnn.py", line 200, in valid_one_epoch
    for x, y in loader:
  File "/app/.pip-target/torch/utils/data/dataloader.py", line 630, in __next__
    data = self._next_data()
           ^^^^^^^^^^^^^^^^^
  File "/app/.pip-target/torch/utils/data/dataloader.py", line 1327, in _next_data
    idx, data = self._get_data()
                ^^^^^^^^^^^^^^^^
  File "/app/.pip-target/torch/utils/data/dataloader.py", line 1283, in _get_data
    success, data = self._try_get_data()
                    ^^^^^^^^^

KeyboardInterrupt: 

In [8]:
# Run parallel XGB FE+CV on CPU while CNN trains
import sys, subprocess, time, os

cmd = [sys.executable, 'train_xgb.py',
       '--out_dir', 'outputs_xgb_v2',
       '--folds', '5',
       '--seed', '42',
       '--n_jobs', str(max(1, (os.cpu_count() or 36) - 4)),
       '--chunksize', '64']
print('> ', ' '.join(cmd), flush=True)
t0 = time.time()
ret = subprocess.run(cmd)
print('XGB return code:', ret.returncode, 'elapsed:', round(time.time()-t0,1), 's', flush=True)

>  /usr/bin/python3.11 train_xgb.py --out_dir outputs_xgb_v2 --folds 5 --seed 42 --n_jobs 32 --chunksize 64


Found train files: 54000, test files: 6000
Featurizing train...
  Parallel FE: n_jobs=32 chunksize=64


  featurizing 5000/54000 elapsed 14.6s


  featurizing 10000/54000 elapsed 24.9s


  featurizing 15000/54000 elapsed 36.3s


...

[0]	train-auc:0.56695	valid-auc:0.49176


[100]	train-auc:0.91931	valid-auc:0.50443
[101]	train-auc:0.92035	valid-auc:0.50444
  Fold 3 best_iteration=27 best_score=0.5099395223046772
XGB Fold 5/5 train=43200 val=10800


[0]	train-auc:0.56854	valid-auc:0.51142


[79]	train-auc:0.90176	valid-auc:0.50585
  Fold 4 best_iteration=5 best_score=0.5229832800483719
XGB OOF AUC: 0.511798
Wrote submission_xgb.csv


XGB return code: 0 elapsed: 150.2 s


In [11]:
# Single-fold CNN sanity check (3ch diffs, pairing B, no aug)
import sys, subprocess, time

cmd = [sys.executable, 'train_cnn.py',
       '--folds', '1',
       '--force_pairing', 'B',
       '--disable_aug',
       '--channels', '3ch',
       '--model_name', 'resnet18',
       '--batch_size', '160',
       '--epochs', '12',
       '--image_size', '256',
       '--num_workers', '8',
       '--lr', '3e-4',
       '--weight_decay', '1e-4',
       '--out_dir', 'outputs_cnn_sanity',
       '--seed', '42']
print('> ', ' '.join(cmd), flush=True)
t0 = time.time()
ret = subprocess.run(cmd)
print('Return code:', ret.returncode, 'elapsed:', round(time.time()-t0,1), 's', flush=True)

>  /usr/bin/python3.11 train_cnn.py --folds 1 --force_pairing B --disable_aug --channels 3ch --model_name resnet18 --batch_size 160 --epochs 12 --image_size 256 --num_workers 8 --lr 3e-4 --weight_decay 1e-4 --out_dir outputs_cnn_sanity --seed 42


Device: cuda


Found train files: 54000, test files: 6000
Using force_pairing=B
--- Fold 1/1 ---


Fold 0 Epoch 1/12 lr=0.000100


  batch 1: running pos ratio=0.500


...

  batch 200: running pos ratio=0.500
  iter 200/270 loss=0.0274 elapsed=11.0s


Fold 0 Epoch 5: train_loss=0.0265 val_auc=0.55573
Early stopping at epoch 5
Fold 0 best AUC: 0.57617 saved to outputs_cnn_sanity/model_fold0.pt


Fold 0 done in 213.3s
OOF AUC: 0.503025


Wrote submission.csv


Return code: 0 elapsed: 218.3 s


In [13]:
# Single-fold CNN sanity check (6ch, pairing B, no aug, no detrend, EMA+clip active in script)
import sys, subprocess, time

cmd = [sys.executable, 'train_cnn.py',
       '--folds', '1',
       '--force_pairing', 'B',
       '--disable_aug',
       '--channels', '6ch',
       '--model_name', 'resnet18',
       '--batch_size', '192',
       '--epochs', '12',
       '--image_size', '256',
       '--num_workers', '8',
       '--lr', '3e-4',
       '--weight_decay', '1e-4',
       '--detrend', 'none',
       '--out_dir', 'outputs_cnn_sanity_6ch_nodetrend',
       '--seed', '42']
print('> ', ' '.join(cmd), flush=True)
t0 = time.time()
ret = subprocess.run(cmd)
print('Return code:', ret.returncode, 'elapsed:', round(time.time()-t0,1), 's', flush=True)

>  /usr/bin/python3.11 train_cnn.py --folds 1 --force_pairing B --disable_aug --channels 6ch --model_name resnet18 --batch_size 192 --epochs 12 --image_size 256 --num_workers 8 --lr 3e-4 --weight_decay 1e-4 --detrend none --out_dir outputs_cnn_sanity_6ch_nodetrend --seed 42


Device: cuda


Found train files: 54000, test files: 6000
Using force_pairing=B
--- Fold 1/1 ---


Fold 0 Epoch 1/12 lr=0.000100


  batch 1: running pos ratio=0.500


...

  batch 100: running pos ratio=0.500
  iter 100/225 loss=0.0095 elapsed=17.2s


  batch 200: running pos ratio=0.500
  iter 200/225 loss=0.0089 elapsed=15.7s


Fold 0 Epoch 8: train_loss=0.0087 val_auc=0.55676
Fold 0 Epoch 9/12 lr=0.000060


  batch 1: running pos ratio=0.500


  batch 100: running pos ratio=0.500
  iter 100/225 loss=0.0046 elapsed=17.2s


In [14]:
# CNN detrending ablation: single-fold, short runs (3-4 epochs), no aug
import sys, subprocess, time

def run_cmd(cmd):
    print('> ', ' '.join(cmd), flush=True)
    t0 = time.time()
    ret = subprocess.run(cmd)
    print('Return code:', ret.returncode, 'elapsed:', round(time.time()-t0,1), 's', flush=True)

# Run 1: detrend=col
cmd_col = [sys.executable, 'train_cnn.py',
           '--folds', '1',
           '--force_pairing', 'B',
           '--disable_aug',
           '--channels', '6ch',
           '--model_name', 'resnet18',
           '--batch_size', '192',
           '--epochs', '4',
           '--image_size', '256',
           '--num_workers', '8',
           '--lr', '3e-4',
           '--weight_decay', '1e-4',
           '--detrend', 'col',
           '--out_dir', 'outputs_cnn_sanity_6ch_col',
           '--seed', '42']
run_cmd(cmd_col)

# Run 2: detrend=row
cmd_row = [sys.executable, 'train_cnn.py',
           '--folds', '1',
           '--force_pairing', 'B',
           '--disable_aug',
           '--channels', '6ch',
           '--model_name', 'resnet18',
           '--batch_size', '192',
           '--epochs', '4',
           '--image_size', '256',
           '--num_workers', '8',
           '--lr', '3e-4',
           '--weight_decay', '1e-4',
           '--detrend', 'row',
           '--out_dir', 'outputs_cnn_sanity_6ch_row',
           '--seed', '42']
run_cmd(cmd_row)

>  /usr/bin/python3.11 train_cnn.py --folds 1 --force_pairing B --disable_aug --channels 6ch --model_name resnet18 --batch_size 192 --epochs 4 --image_size 256 --num_workers 8 --lr 3e-4 --weight_decay 1e-4 --detrend col --out_dir outputs_cnn_sanity_6ch_col --seed 42


Device: cuda


Found train files: 54000, test files: 6000
Using force_pairing=B
--- Fold 1/1 ---


Fold 0 Epoch 1/4 lr=0.000100


  batch 1: running pos ratio=0.500


...

  batch 200: running pos ratio=0.500
  iter 200/225 loss=0.2327 elapsed=20.5s


Fold 0 Epoch 4: train_loss=0.2316 val_auc=0.53096
Fold 0 best AUC: 0.53096 saved to outputs_cnn_sanity_6ch_row/model_fold0.pt


Fold 0 done in 285.0s
OOF AUC: 0.501217


Wrote submission.csv


Return code: 0 elapsed: 292.8 s


In [15]:
# Sanity: EfficientNet-B0 6ch, percentile norm, no aug, detrend=none
import sys, subprocess, time
cmd = [sys.executable, 'train_cnn.py',
       '--folds', '1',
       '--force_pairing', 'B',
       '--disable_aug',
       '--channels', '6ch',
       '--model_name', 'tf_efficientnet_b0_ns',
       '--batch_size', '192',
       '--epochs', '4',
       '--image_size', '256',
       '--num_workers', '8',
       '--lr', '1e-3',
       '--weight_decay', '1e-4',
       '--detrend', 'none',
       '--use_percentile_norm',
       '--out_dir', 'outputs_cnn_effb0_sanity',
       '--seed', '42']
print('> ', ' '.join(cmd), flush=True)
t0 = time.time()
ret = subprocess.run(cmd)
print('Return code:', ret.returncode, 'elapsed:', round(time.time()-t0,1), 's', flush=True)

>  /usr/bin/python3.11 train_cnn.py --folds 1 --force_pairing B --disable_aug --channels 6ch --model_name tf_efficientnet_b0_ns --batch_size 192 --epochs 4 --image_size 256 --num_workers 8 --lr 1e-3 --weight_decay 1e-4 --detrend none --use_percentile_norm --out_dir outputs_cnn_effb0_sanity --seed 42


Device: cuda


Found train files: 54000, test files: 6000
Using force_pairing=B
--- Fold 1/1 ---


/app/.pip-target/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name tf_efficientnet_b0_ns to current tf_efficientnet_b0.ns_jft_in1k.
  model = create_fn(


Fold 0 Epoch 1/4 lr=0.000200


...

  iter 200/225 loss=0.0168 elapsed=29.4s


Fold 0 Epoch 4: train_loss=0.0162 val_auc=0.50120
Early stopping at epoch 4
Fold 0 best AUC: 0.50456 saved to outputs_cnn_effb0_sanity/model_fold0.pt


Fold 0 done in 355.4s
OOF AUC: 0.500162


Wrote submission.csv


Return code: 0 elapsed: 363.9 s


In [16]:
# Sanity: EffNet-B0 6ch, per-image std norm, no aug, detrend=none
import sys, subprocess, time
cmd = [sys.executable, 'train_cnn.py',
       '--folds', '1',
       '--force_pairing', 'B',
       '--disable_aug',
       '--channels', '6ch',
       '--model_name', 'tf_efficientnet_b0_ns',
       '--batch_size', '192',
       '--epochs', '4',
       '--image_size', '256',
       '--num_workers', '8',
       '--lr', '1e-3',
       '--weight_decay', '1e-4',
       '--detrend', 'none',
       '--per_image_std',
       '--out_dir', 'outputs_cnn_effb0_perimgstd_sanity',
       '--seed', '42']
print('> ', ' '.join(cmd), flush=True)
t0 = time.time()
ret = subprocess.run(cmd)
print('Return code:', ret.returncode, 'elapsed:', round(time.time()-t0,1), 's', flush=True)

>  /usr/bin/python3.11 train_cnn.py --folds 1 --force_pairing B --disable_aug --channels 6ch --model_name tf_efficientnet_b0_ns --batch_size 192 --epochs 4 --image_size 256 --num_workers 8 --lr 1e-3 --weight_decay 1e-4 --detrend none --per_image_std --out_dir outputs_cnn_effb0_perimgstd_sanity --seed 42


Device: cuda


Found train files: 54000, test files: 6000
Using force_pairing=B
--- Fold 1/1 ---


/app/.pip-target/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name tf_efficientnet_b0_ns to current tf_efficientnet_b0.ns_jft_in1k.
  model = create_fn(


Fold 0 Epoch 1/4 lr=0.000200


...

  iter 200/225 loss=0.0144 elapsed=29.4s


Fold 0 Epoch 4: train_loss=0.0140 val_auc=0.51599
Fold 0 best AUC: 0.51599 saved to outputs_cnn_effb0_perimgstd_sanity/model_fold0.pt


Fold 0 done in 340.9s
OOF AUC: 0.500619


Wrote submission.csv


Return code: 0 elapsed: 349.8 s


In [ ]:
# Sanity: ConvNeXt-Tiny 6ch, percentile norm, no aug, detrend=none, no EMA
import sys, subprocess, time
cmd = [sys.executable, 'train_cnn.py',
       '--folds', '1',
       '--force_pairing', 'B',
       '--disable_aug',
       '--channels', '6ch',
       '--model_name', 'convnext_tiny',
       '--batch_size', '160',
       '--epochs', '4',
       '--image_size', '256',
       '--num_workers', '8',
       '--lr', '5e-4',
       '--weight_decay', '1e-4',
       '--detrend', 'none',
       '--use_percentile_norm',
       '--no_ema',
       '--out_dir', 'outputs_cnn_convnext_tiny_sanity',
       '--seed', '42']
print('> ', ' '.join(cmd), flush=True)
t0 = time.time()
ret = subprocess.run(cmd)
print('Return code:', ret.returncode, 'elapsed:', round(time.time()-t0,1), 's', flush=True)

>  /usr/bin/python3.11 train_cnn.py --folds 1 --force_pairing B --disable_aug --channels 6ch --model_name convnext_tiny --batch_size 160 --epochs 4 --image_size 256 --num_workers 8 --lr 5e-4 --weight_decay 1e-4 --detrend none --use_percentile_norm --no_ema --out_dir outputs_cnn_convnext_tiny_sanity --seed 42


Device: cuda


Found train files: 54000, test files: 6000
Using force_pairing=B
--- Fold 1/1 ---
